# Labeling the data (Archival)

Most of the data scraped from the Wikipedia tables is not fully structured. The battle outcomes are not clearly paired with sets of belligerents, and the forces and casualties fields are not consistently formatted.

To deal with this, ethanarsht hand-labeled most of the battles in Excel. (Mad props; this is a ton of data to deal with in that manner.) This notebook preserves most of that workflow, with minor edits to the `pandas` code to bring it up to speed with the 2020 library and Python 3.7. 

It is *not* really meant to be run from scratch - to recreate the workflow exactly, you'd need to label the data yourself in an external file editor. Instead, I'm preserving it as a record of some of the data manipulation steps that were taken intermediately.

## Imports and config

In [ ]:
from functools import reduce
from operator import iand, ior

import pandas as pd

from sklearn.linear_model import LogisticRegression

## Add results

In [ ]:
old_result_column = pd.read_csv('./data/result_column.csv', names=['Battle', 'leader', 'pos', 'Result', 'VorD'])
old_result_column['Battle'] = old_result_column.Battle.map(lambda s: s.replace('_', ' '))

In [ ]:
results_with_outcomes = pd.merge(results, old_result_column, how='outer', on=['Battle', 'leader', 'pos', 'Result'])
results_with_outcomes.dropna(subset=('VorD', ), inplace=True)
results_with_outcomes.drop_duplicates(subset=('Battle', 'Date', 'VorD'), inplace=True)

results_with_outcomes.sample(n=5)

In [ ]:
# Old results merge
df_more_results = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'pos', 'Result'])
df_more_results = df_more_results[df_more_results.VorD.isnull()]
df_more_results = df_more_results.drop_duplicates(subset = ['Battle', 'Date', 'VorD'])
df_more_results.to_csv('./data/adding_results_10.21.csv', encoding = 'utf-8')

df_more_results.head()

In [ ]:
# More data inspection?

df_vl = pd.merge(results, old_result_column, how='outer', on=['Battle', 'leader', 'pos', 'Result'])
df_vl.sort_values('VorD', inplace=True)

df_vl[df_vl.belligerent == 'Ambroise de Loré']
df_vl[df_vl.belligerent == 'Saracens']

df_vl.to_csv('./data/additional_results.csv')

## Add strengths

In [ ]:
# Prepare for hand-labeling strengths
df_strength = pd.read_csv('./data/additional_results.csv', encoding='utf-8', index_col=0)
df_strength.drop(['Location', 'Result', 'inflicted', 'taken', 'opp'], axis=1, inplace=True)
df_strength.sort_values('Battle', inplace=True)
df_strength.reset_index(drop=True, inplace=True)
df_strength.drop_duplicates(['own', 'pos'], keep='first', inplace=True)

df_strength['own'] = df_strength.own.str.replace('\r',' ').str.replace('\n', ' ').replace('\n', ' ')
df_strength.to_csv('strength_entry.csv', encoding='utf-8')

In [ ]:
df_strength = pd.read_excel('./data/strength_entry.xlsx', encoding='utf-8')
df_strength['Battle'] = df_strength.Battle.str.replace(u"â€“", u'–').replace('_', ' ')
df_strength.sample(n=5)

In [ ]:
# NB: This may be broken as is. 
#     You may need to pass ./data/scraped_battles_data.csv 
#     instead if trying to recreate this flow from scratch.
#     (And then posssibly fiddle some more.)

df_battles = pd.read_csv('./data/battles_deduped.csv', encoding='utf-8', index_col=0)
df_battles.drop(['taken', 'inflicted'], axis=1, inplace=True)

In [ ]:
results_with_strengths = pd.merge(results_with_outcomes, df_strength, how='outer', on=['Battle', 'pos'])
results_with_strengths.drop(['own_y', 'Date_y'], axis=1, inplace=True)
results_with_strengths.rename(columns={'Date_x': 'Date','own_x': 'own', 'belligerent_x': 'belligerent'}, inplace=True)
results_with_strengths.sample(n=5)

In [ ]:
# If you are trying to recreate this workflow, get ready for fun! 
# Label it all yourself in Excel.
# (This cell block should run anyway by using cached data; preserving to show workflow)

null_idx = reduce(lambda c, s: iand(c, results_with_strengths[s].isnull()), 
                  ['Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], 
                  results_with_strengths.Infantry.isnull())

df_str_two = results_with_strengths.loc[null_idx]

df_str_two.drop_duplicates(['Battle', 'Date', 'Location', 'Result', 'pos'], inplace=True)

df_merger = df_strength.drop(['Date', 'belligerent', 'own'], axis=1)
df_merger.drop_duplicates(inplace=True)

df_str_two = pd.merge(df_str_two, df_merger, on=['Battle', 'pos'], how='outer')

df_str_two.dropna(subset=['Date', 'Location', 'Result'], inplace=True)
df_str_two.dropna(subset=['VorD'], inplace=True)

# df_str_two.to_csv('strength_entry_two.csv')
df_str_two.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], axis=1, inplace=True)

df_partial = pd.read_excel('PARTIAL_strength_entry_two.xlsx', encoding='utf-8')
df_partial = df_partial.drop(['Date', 'Location', 'Result', 'belligerent','own', 'Column1'],axis = 1)
df_finish = pd.merge(df_str_two, df_partial, on = ['Battle', 'pos', 'VorD'], how = 'outer')
drop_list = list(df_finish[df_finish['Date'].isnull()].index)
df_finish = df_finish.drop(drop_list)
df_finish.to_csv('last_strength_entry.csv', encoding='utf-8')

df_more_strength = pd.read_excel('last_strength_entry.xlsx')
df_more_strength = pd.merge(df_more_strength, df_str_two, on = ['Battle', 'pos'], how = 'outer').drop(['own_y', 'Infantry_y', 'Cavalry_y', 'Artillery_y', 'Ships_y', 'Airforce_y', 'Special_y', 'Date_y', 'Location_y', 'Result_y', 'belligerent_y'], axis = 1).dropna(subset = ['Date_x'])
df_more_strength.columns = df_more_strength.columns.str.replace('_x', '')

df_fill_two = pd.merge(df_battles, df_more_strength, on = ['Battle', 'pos'], how='outer')
df_fill_two.drop(['Date_y', 'Location_y', 'belligerent_y', 'Result_y', 'own_y', 'opp_y'], axis=1, inplace=True)

df_fill_two.columns = df_fill_two.columns.str.replace('_x', '')
df_fill_two = df_fill_two[df_fill_two['Infantry'].notnull() | df_fill_two['Cavalry'].notnull() | df_fill_two['Artillery'].notnull() | df_fill_two['Ships'].notnull() | df_fill_two['Airforce'].notnull() | df_fill_two['Special'].notnull()]
df_some_strength = df_some_strength.loc[df_some_strength['Infantry'].notnull() | df_some_strength['Cavalry'].notnull() | df_some_strength['Artillery'].notnull() | df_some_strength['Ships'].notnull() | df_some_strength['Airforce'].notnull() | df_some_strength['Special'].notnull()]
df_all_strength = pd.concat([df_fill_two, df_some_strength]).reset_index(drop = True)
df_char = df_all_strength[df_all_strength.Date.isnull()]

df_char.to_excel('special_character_fix.xlsx')
df_all_strength.to_excel('all_strength_probably.xlsx')

## Logistic regression

### Configure swapped data

In [ ]:
UNIT_TYPES = ('Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special')

In [ ]:
df_strength_all = pd.read_excel('./data/all_strength_probably.xlsx')
df_strength_all.loc[9988, 'Infantry'] = 2000
df_strength_all.drop_duplicates(inplace=True)

print(len(df_strength_all))
df_strength_all.sample(n=5)

In [ ]:
df_left = df_strength_all.loc[df_strength_all.pos == 'L']
df_left.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own'], axis=1, inplace=True)
df_left.drop_duplicates(subset=['Battle'], inplace=True)
df_left.dropna(subset=['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how='all', inplace=True)

df_right = df_strength_all.loc[df_strength_all.pos == 'R']
df_right.drop_duplicates(subset=['Battle'], inplace=True)
df_right.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'VorD'], axis=1, inplace=True)
df_right.dropna(subset=['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how='all', inplace=True)

df_model = pd.merge(df_left, df_right, on='Battle', how='outer')
df_model.dropna(subset=['pos_x', 'pos_y'], inplace=True)
df_model.reset_index(drop=True, inplace=True)

print(len(df_model))
df_model.sample(n=5)

In [ ]:
df_v = df_model.loc[df_model.VorD == 'V']
df_switch = df_model.sample(n=374).copy()
df_switch['VorD'] = 'D'

def swap_cols(df, col):
    """Swap x and y column contents."""
    df[f"{col}_x"], df[f"{col}_y"] = df[f"{col}_y"], df[f"{col}_x"]
    return df

for unit_type in UNIT_TYPES:
    _ = swap_cols(df_switch, unit_type)

df_model.drop(df_switch.index, inplace=True)
df_model = pd.concat([df_model, df_switch])

# ?
df_model.loc[2378, 'Infantry_y'] = 25000
df_model.loc[2363, 'Infantry_x'] = 500

df_model = df_model.loc[df_model.Infantry_x != 1.0]
df_model = df_model.loc[df_model.Infantry_y != 1.0]

df_model.fillna(0, inplace=True)

for unit_type in UNIT_TYPES:
    x, y = df_model[f"{unit_type}_x"], df_model[f"{unit_type}_y"]
    df_model[f"{unit_type}_diff"] = (x - y) / (x + y)

In [ ]:
df_diff = df_model[['VorD', 'Battle'] + [f"{unit_type}_diff" for unit_type in UNIT_TYPES]]

df_diff = df_diff.loc[df_diff.VorD != 'I']
df_diff.drop(174, inplace=True)
df_diff.fillna(0, inplace=True)

df_x = df_diff.iloc[:, 2:-1]
df_x.drop('Artillery_diff', axis=1, inplace=True)

df_y = df_diff.iloc[:, 0:1]
df_y['VorD'] = df_y.VorD.replace('V', 1).replace('D', 0).replace('L', 0).replace('v', 1).astype('float')

### Fit and save logistic regression model

In [ ]:
lr = LogisticRegression()
lr.fit(df_x, df_y)

with open('./data/saved_logistic_regression.pkl', 'wb') as f:
    pickle.dump(lr, f)

lr.score(df_x, df_y)

## Final changes to strengths table

NB: I'm not sure why this was done after computing the logistic regression instead of before?

In [ ]:
# Hand corrections
# Would be nice if these were documented?
# (They were all hand-inputted anyway, I believe...)
df_strength_all.loc[60, 'VorD'] = 'V'
df_strength_all.loc[61, 'VorD'] = 'V'
df_strength_all.loc[3479:3485, 'VorD'] = 'V'
df_strength_all.loc[3485, 'VorD'] = 'D'
df_strength_all.loc[3802:3803, 'VorD'] = 'V'
df_strength_all.loc[3804:3807, 'VorD'] = 'D'
df_strength_all.loc[4142:4143, 'VorD'] = 'D'

# Corrections for inconsistent notations
df_strength_all['VorD'] = df_strength_all.VorD.str.replace('L', 'D')
df_strength_all['VorD'] = df_strength_all.VorD.str.replace('v', 'V')
df_strength_all['VorD'] = df_strength_all.VorD.str.replace('W', 'V')

# More hand tweaks...
null_fill = df_strength_all.loc[df_strength_all.VorD.isnull()]

null_fill['VorD'] = ['V', 'D', 'D', 'V', 'V', 'D', 'V', 'V', 'V', 
                     'V', 'D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 
                     'V', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D',
                     'D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V', 
                     'D', 'D', 'D', 'D', 'V', 'V', 'D', 'D', 'D', 
                     'D', np.NaN, 'D', np.NaN]

df_strength_all.VorD.loc[null_fill.index] = null_fill['VorD']
df_strength_all = df_strength_all.loc[df_strength_all.belligerent != 'Sweden']

In [ ]:
# Extract years from dates
def get_year(row):
    d = row.Date
    i = row.name
    
    if isinstance(d, str):
        if re.search(r'[12]\d{3}', d):
            year = re.search(r'[12]\d{3}', d).group(0)
            year = int(year)
            
            if 'BC' in d:
                year *= -1
            
            return year
                            
        elif re.search(r'[12]\d{2}', d):
            year = re.search(r'[12]\d{2}', d).group(0)
            year = int(year)
            
            if 'BC' in d:
                year *= -1
                
            return year
    
    return None

df_strength_all['year'] = df_strength_all.apply(get_year, axis=1).astype(float)

In [ ]:
# Only save if you are OK overwriting the original archived data
# df_strength_all.to_csv('./data/current_run.csv', encoding='utf-8')